In [ ]:
import ipywidgets as W
from IPython.display import display, clear_output
from tradle_guesser import load_country_data, best_guesses

# Load data
print("Loading country data...")
centroid_list, distance_df = load_country_data()
countries = sorted(centroid_list["Country"].unique().tolist())
dir_choices = ["", "N", "S", "E", "W", "NE", "NW", "SE", "SW"]
print(f"Loaded {len(countries)} countries")

In [ ]:
rows_box = W.VBox([])  # will hold hint rows

def make_row():
    return W.HBox([
        W.Dropdown(options=countries, description="Country", layout=W.Layout(width="40%")),
        W.BoundedIntText(value=1000, min=0, max=40075, step=1, description="km", layout=W.Layout(width="25%")),
        W.Dropdown(options=dir_choices, value="", description="Dir", layout=W.Layout(width="20%")),
        W.Button(description="−", button_style="", layout=W.Layout(width="8%"))
    ])

def add_row(_=None):
    row = make_row()
    def remove_row(btn):
        rows = list(rows_box.children)
        rows.remove(row)
        rows_box.children = tuple(rows)
    row.children[-1].on_click(remove_row)
    rows_box.children = (*rows_box.children, row)

add_btn = W.Button(description="Add hint", button_style="info")
add_btn.on_click(add_row)
add_row()  # start with one

penalty = W.BoundedFloatText(value=10.0, min=0, max=1e6, step=1.0, description="Direction penalty")
topn    = W.BoundedIntText(value=15, min=1, max=250, step=1, description="Top N")

run_btn = W.Button(description="Compute", button_style="success")
out = W.Output()

def run(_):
    hints = []
    for row in rows_box.children:
        ctry = row.children[0].value
        km   = row.children[1].value
        d    = row.children[2].value or None
        hints.append((ctry, km) if d is None else (ctry, km, d))
    with out:
        clear_output(wait=True)
        if not hints:
            print("Add at least one hint.")
            return
        try:
            res = best_guesses(hints, centroid_list, distance_df, penalty=float(penalty.value))
            display(res.head(int(topn.value)))
        except Exception as e:
            print(f"Error: {e}")

run_btn.on_click(run)

ui = W.VBox([
    W.HTML("<h2>Tradle Geographic Triangulator</h2><p>Enter hints (country, distance km, optional direction).</p>"),
    W.HBox([add_btn, penalty, topn, run_btn]),
    rows_box,
    out
])

display(ui)